In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import sys

module_path = '/kaggle/input/bratsslmsamodel'  # Update this with the correct path
sys.path.append(module_path)

#Import your module
import bratsslmsa



In [ ]:
import sys

module_path = '/kaggle/input/diceioumetrics4ch'  # Update this with the correct path
sys.path.append(module_path)

import diceioumetrics4ch
from diceioumetrics4ch import dice_loss,dice_coefficient,IoUClassMetrics

In [ ]:
!pip install tensorflow_addons

In [ ]:
import numpy as np
import nibabel as nib
import os
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import glob
import tensorflow as tf
from tensorflow_addons.optimizers import AdamW
import keras.optimizers as optim
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger,ReduceLROnPlateau,EarlyStopping
import matplotlib.pyplot as plt
import random
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:


# Define a function to load and preprocess a single volume
def load_and_preprocess_volume(volume_path, mask_path):
    
    # Load the volume and mask for each channel
    t2_volume = nib.load(volume_path[0]).get_fdata()
    t2_volume_scaled = scaler.fit_transform(t2_volume.reshape(-1, t2_volume.shape[-1])).reshape(t2_volume.shape)
    t1ce_volume = nib.load(volume_path[1]).get_fdata() 
    t1ce_volume_scaled = scaler.fit_transform(t1ce_volume.reshape(-1, t1ce_volume.shape[-1])).reshape(t1ce_volume.shape)
    flair_volume = nib.load(volume_path[2]).get_fdata()
    flair_volume_scaled = scaler.fit_transform(flair_volume.reshape(-1, flair_volume.shape[-1])).reshape(flair_volume.shape)
    mask = nib.load(mask_path).get_fdata().astype(np.uint8)
    mask[mask == 4] = 3  # Reassign mask values 4 to 3 or perform any other necessary preprocessing

    # Combine the channels
    combined_x = np.stack([t2_volume_scaled, t1ce_volume_scaled, flair_volume_scaled], axis=-1)
    return combined_x, mask

# Define a data generator
def custom_datagen(volume_path, mask_paths, batch_size):
    
    num_samples = len(volume_path)
    print("volume_path len:",len(volume_path))
    while True:
        for i in range(0, num_samples, batch_size):
            batch_volume_paths = volume_path[i:i + batch_size]
            batch_mask_paths = mask_paths[i:i + batch_size]
            X = []
            Y = []
            for volume_path1, mask_path in zip(batch_volume_paths, batch_mask_paths):
                             
                image, mask = load_and_preprocess_volume(volume_path1, mask_path)

                # Crop to a size divisible by the patch size
                image = image[56:184, 56:184, 13:141]
                mask = mask[56:184, 56:184, 13:141]
                temp_mask= to_categorical(mask, num_classes=4)
               
                X.append(image)
                Y.append(temp_mask)
            
            X = np.array(X)
            Y = np.array(Y)
            
            yield X, Y

# Define the data directory
data_directory = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'

# Create lists of file paths
t2_list = sorted(glob.glob(os.path.join(data_directory,'*/','*_t2.nii'), recursive=True))
remove = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/BraTS20_Training_355_t2.nii' 
t2_list = [item for item in t2_list if item != remove]
t1ce_list = sorted(glob.glob(os.path.join(data_directory, '*/','*_t1ce.nii'), recursive=True))
remove1 = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/BraTS20_Training_355_t1ce.nii'
t1ce_list = [item for item in t1ce_list if item != remove1]
flair_list = sorted(glob.glob(os.path.join(data_directory,'*/','*_flair.nii'), recursive=True))
remove2 = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/BraTS20_Training_355_flair.nii'
flair_list = [item for item in flair_list if item != remove2]
mask_list = sorted(glob.glob(os.path.join(data_directory, '*/','*_seg.nii'), recursive=True))

# Create data generators
batch_size = 2
train_image_paths, val_image_paths, train_mask_paths, val_mask_paths = train_test_split( list(zip(t2_list, t1ce_list, flair_list)), mask_list, test_size=0.4, random_state=42)
train_image_paths, test_image_paths, train_mask_paths, test_mask_paths = train_test_split( train_image_paths, train_mask_paths, test_size=0.2, random_state=42)
print("train_image_paths =",len(train_image_paths))
print("test_image_paths =",len(test_image_paths))
print("val_image_paths =",len(val_image_paths))

train_dataset = custom_datagen(train_image_paths, train_mask_paths, batch_size)
val_dataset = custom_datagen(val_image_paths, val_mask_paths, batch_size)
test_dataset = custom_datagen(test_image_paths, test_mask_paths, batch_size)
print(train_dataset, val_dataset, test_dataset)

In [ ]:
# # Load and preprocess a single volume from the test dataset
# for data in test_dataset:
#     test_img_input, test_mask = data
#     print("test_img_input.shape",test_img_input.shape)
#     print("test_mask.shape",test_mask.shape)
    
#     #Here, test_img_input is the input image, and test_mask is the corresponding mask
#     test_mask_argmax=np.argmax(test_mask, axis=4)
#     print("test_mask_argmax.shape",test_mask_argmax.shape)
    
# #     test_img = np.expand_dims(test_img_input, axis=4)
# #     print("test_img.shape",test_img.shape)
    
#     # Predict the mask for the input image
#     test_prediction = model.predict(test_img_input)
#     print("test_prediction.shape",test_prediction.shape)
#     # Assuming your model's output has multiple channels, get the argmax of the last dimension
#     test_prediction_argmax = np.argmax(test_prediction, axis=4)
    
#     print("test_prediction=",test_prediction.shape)
#     print("test_prediction_argmax=",np.unique(test_prediction_argmax)) 
#     print("test_mask_argmax=",np.unique(test_mask_argmax))
#     print("test_mask=",np.unique(test_mask))
#     print("test_prediction=",np.unique(test_prediction))
#     n_slice = 85
#     plt.figure(figsize=(12, 8))
#     plt.subplot(231)
#     plt.title('Testing Image')
#     plt.imshow(test_img_input[0,:,:,n_slice,2], cmap='gray')
#     plt.subplot(232)
#     plt.title('Testing Label')
#     plt.imshow(test_mask_argmax[0,:,:,n_slice])
#     plt.subplot(233)
#     plt.title('Prediction on test image')
#     plt.imshow(test_prediction_argmax[0,:,:, n_slice])
#     plt.show()
#     break  # Stop after predicting one set

# # test_prediction_argmax will contain the predicted mask for the input image


In [ ]:
import tensorflow as tf
from tensorflow_addons.optimizers import AdamW
from keras.utils import custom_object_scope
from bratsslmsa import ShortTermMemoryAttentionBlock, LongTermMemoryAttentionBlock  # Import your custom layers

modelpath = '/kaggle/input/brats-slmsa-7diceloss/brats_slmsa_7diceloss.hdf5'

if os.path.exists(modelpath):
    # Load the model with custom layer registration
    with custom_object_scope({
        'ShortTermMemoryAttentionBlock': ShortTermMemoryAttentionBlock,
        'LongTermMemoryAttentionBlock': LongTermMemoryAttentionBlock, 
        'dice_loss':dice_loss,
        'dice_coefficient': dice_coefficient,
        'IoUClassMetrics' : IoUClassMetrics
            }):
        model = load_model(modelpath)
else:
    print(f"The model file '{modelpath}' does not exist.")

 

In [ ]:
LR = 1e-4
epoch=20
import keras.optimizers as optim
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger,ReduceLROnPlateau,EarlyStopping
from tensorflow_addons.optimizers import AdamW


model_hdf5_path = "/kaggle/working/brats_slmsa_mordat_diceloss.hdf5"
csv_path = '/kaggle/working/brats_slmsa_mordat_diceloss.csv'

steps_per_epoch = len(train_image_paths)//batch_size
val_steps_per_epoch = len(val_image_paths)//batch_size


from  bratsslmsa import slm_sa
num_classes = 4
iou_metrics = IoUClassMetrics() 
#total_steps = steps_per_epoch * epoch
#total_loss = TotalLoss3D()
weight_decay = 1e-5  # desired weight decay
metrics =[dice_coefficient,iou_metrics]
# Compile your model with the AdamW optimizer
optimizer = AdamW(learning_rate=LR, weight_decay=weight_decay)

model.compile(optimizer=optimizer,
              loss=dice_loss,
              metrics=metrics)
callbacks = [
    ModelCheckpoint(model_hdf5_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-9, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]


In [ ]:
history=model.fit(train_dataset,
          steps_per_epoch=steps_per_epoch,
          epochs=20,
          verbose=1,
          validation_data= val_dataset,
          validation_steps=val_steps_per_epoch,
          callbacks=callbacks)

model.save(model_hdf5_path)
